<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB37_Custom_Agent_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB37 : 1er custom Tool pour Agent :

**Objectif:** :
- Concevoir et implémenter un outil personnalisé réutilisable (nom, description, schéma d'entrée, fonction)
- Validation
- Gestion d'erreurs
- Mise en cache
- Exposition à un agent.

**Durée estimée:** 15–20 minutes

**Livrable:** Un module Python avec l'outil + un agent de démonstration qui l'appelle.

---

## Étape 1: Setup

Installation des dépendances nécessaires.

In [1]:
# Installation des packages
!pip install -q langchain langchain-openai openai python-dotenv requests beautifulsoup4 pydantic cachetools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 1.7 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


## Étape 2: Définition des spécifications du "tool" :

Définition de l'interface de l'outil :

- **Name:** UrlSummarizer
- **Description:** "Récupère une page web, extrait le texte principal et retourne un résumé concis."
- **Inputs:** `{ "url": str, "max_words": int = 120 }`
- **Outputs:** chaîne de résumé courte

In [5]:
from pydantic import BaseModel, HttpUrl, PositiveInt

# Schéma d'entrée pour l'outil
class UrlSummarizeInput(BaseModel):
    """Schéma de validation pour l'outil UrlSummarizer."""
    url: HttpUrl
    max_words: PositiveInt = 120

# Test du schéma
test_input = UrlSummarizeInput(url="https://1data-hexagone.fr/", max_words=100)
print(f"✓ Schéma défini: {test_input}")

✓ Schéma défini: url=HttpUrl('https://1data-hexagone.fr/') max_words=100


## Étape 3: Implémentation du coeur de l'outil :

### Partie 1: Fonctions utilitaires

In [6]:
import re
import time
import requests
from bs4 import BeautifulSoup
from typing import Optional
from pydantic import ValidationError

# -------- Fonctions utilitaires --------

def clean_text(txt: str) -> str:
    """Nettoie le texte en supprimant les espaces multiples."""
    txt = re.sub(r'\s+', ' ', txt).strip()
    return txt

def extract_main_text(html: str) -> str:
    """Extrait le texte principal d'une page HTML."""
    soup = BeautifulSoup(html, "html.parser")

    # Heuristique: privilégier <article>, sinon joindre tous les <p>
    article = soup.find("article")
    if article:
        text = " ".join(p.get_text(" ", strip=True) for p in article.find_all("p"))
    else:
        text = " ".join(p.get_text(" ", strip=True) for p in soup.find_all("p"))

    return clean_text(text)

def http_get(url: str, retries: int = 3, backoff: float = 1.5, timeout: int = 15) -> requests.Response:
    """Effectue une requête HTTP avec retry pour les échecs transitoires."""
    last_err = None
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, timeout=timeout, headers={"User-Agent": "AgenticAI/1.0"})
            r.raise_for_status()
            return r
        except requests.RequestException as e:
            last_err = e
            if attempt == retries:
                raise
            time.sleep(backoff ** attempt)
    raise last_err

print("✓ Fonctions utilitaires définies")

✓ Fonctions utilitaires définies


### Partie 2: Système de cache

In [8]:
from cachetools import TTLCache, cached

# Cache avec TTL de 10 minutes
cache = TTLCache(maxsize=256, ttl=60 * 10)

@cached(cache)
def fetch_and_extract(url: str) -> str:
    """Récupère et extrait le contenu d'une URL (avec cache)."""
    resp = http_get(url)
    return extract_main_text(resp.text)

print(f"✓ Système de cache initialisé (TTL: 10 min, max: 256 entrées)")

✓ Système de cache initialisé (TTL: 10 min, max: 256 entrées)


### Partie 3: Fonction principale de l'outil

In [9]:
def summarize_url(url: str, max_words: int = 120) -> str:
    """Récupère une URL, extrait le texte et retourne un résumé."""

    # Validation des entrées
    try:
        args = UrlSummarizeInput(url=url, max_words=max_words)
    except ValidationError as ve:
        return f"Erreur de validation: {ve}"

    # Récupération et extraction du contenu
    try:
        text = fetch_and_extract(str(args.url))
        if not text or len(text.split()) < 30:
            return "Désolé, impossible de trouver suffisamment de contenu lisible sur cette page."
    except Exception as e:
        return f"Échec de la récupération ou du parsing de l'URL: {e}"

    # Résumé heuristique (compression légère)
    words = text.split()
    if len(words) <= args.max_words:
        return " ".join(words)

    # Troncature consciente des phrases
    out = []
    count = 0
    for sentence in re.split(r'(?<=[.!?])\s+', text):
        sw = sentence.split()
        if count + len(sw) > args.max_words:
            break
        out.append(sentence)
        count += len(sw)

    if not out:
        out = words[:args.max_words]
        return " ".join(out) + " ..."

    return " ".join(out)

# Test de la fonction
print("✓ Fonction summarize_url définie")
print("\nTest avec example.com:")
result = summarize_url("https://example.com", max_words=50)
print(f"Résultat: {result[:100]}...")

✓ Fonction summarize_url définie

Test avec example.com:
Résultat: Désolé, impossible de trouver suffisamment de contenu lisible sur cette page....


In [10]:
# Test de la fonction avec un site existant :
print("✓ Fonction summarize_url définie")
print("\nTest avec example.com:")
result = summarize_url("https://1data-hexagone.fr/", max_words=50)
print(f"Résultat: {result[:100]}...")

✓ Fonction summarize_url définie

Test avec example.com:
Résultat: MaÃ®trisez les technologies leaders du Big Data Chez 1Data-Hexagone, nous formons les professionnels...


## Étape 4: Fournissons l'outil en tant qu'outil LangChain : )

Création des outils LangChain.

In [11]:
from langchain_core.tools import Tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# Outil UrlSummarizer
url_tool = Tool(
    name="UrlSummarizer",
    func=lambda q: summarize_url(**q) if isinstance(q, dict) else summarize_url(q),
    description=(
        "Récupère et résume une page web. "
        "Input JSON avec clés: url (string), max_words (int, optionnel, défaut 120). "
        "Utiliser pour extraire le contenu principal et obtenir un résumé concis."
    )
)

# Outil Calculator (pour la variété)
def calculator(expr: str):
    """Évalue une expression mathématique."""
    try:
        return str(eval(expr))
    except Exception as e:
        return f"Erreur: {e}"

calc_tool = Tool(
    name="Calculator",
    func=calculator,
    description="Évalue des expressions mathématiques comme '12*(7+3)'."
)

# Initialisation du LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Création de l'agent avec create_agent
agent = create_agent(
    model=llm,
    tools=[url_tool, calc_tool],
    system_prompt="You are a helpful assistant. Use the tools available to answer questions accurately."
)

print("✓ Agent initialisé avec 2 outils: UrlSummarizer et Calculator")

✓ Agent initialisé avec 2 outils: UrlSummarizer et Calculator


## Étape 5: Essai avec des requêtes réelles :

### Test 1: Résumer une page web

In [16]:
# Test 1 avec l'URL summarizer
response = agent.invoke({
    "messages": [{"role": "user", "content": "Résume cette page: https://1data-hexagone.fr/"}]
})
print(response['messages'][-1].content)

La page de 1Data-Hexagone présente des formations destinées aux professionnels sur les technologies du Big Data. Elle met l'accent sur l'apprentissage des écosystèmes Hadoop, des plateformes de streaming en temps réel, et des solutions d'intelligence artificielle modernes. Les formations sont pratiques, orientées vers les métiers, et dispensées par des experts confirmés.


### Test 2: Utiliser Calculator

In [17]:
# Test 2 avec la calculatrice
response = agent.invoke({
    "messages": [{"role": "user", "content": "Calcule 12*(7+3)"}]
})
print(response['messages'][-1].content)


Le résultat de \( 12 \times (7 + 3) \) est 120.


### Test 3: Requête combinée

In [22]:
# Test 3: Requête combinée
response = agent.invoke({
    "messages": [{"role": "user", "content": "Calcule l'âge du capitaine Némo au moment de sa disparition dans L'île Mystérieuse + 12 et ensuite résume cette page: https://comparia.beta.gouv.fr/"}]
})
print(response['messages'][-1].content)


L'âge du capitaine Némo au moment de sa disparition dans "L'île Mystérieuse" est de 70 ans. En ajoutant 12, cela donne un total de **82 ans**.

Concernant la page https://comparia.beta.gouv.fr/, voici un résumé :

Le site "compar:IA" est un comparateur d'IA conversationnelles qui permet aux utilisateurs de discuter avec deux IA de manière anonyme et d'évaluer leurs réponses. Les données échangées sont utilisées à des fins de recherche. Les utilisateurs peuvent interagir aussi longtemps qu'ils le souhaitent, et les modèles d'IA sont dévoilés après la discussion. Le site vise à sensibiliser les citoyens aux enjeux de l'IA générative et à encourager le développement d'un esprit critique. Les utilisateurs peuvent tester différents modèles d'IA, qu'ils soient propriétaires ou non, et de différentes tailles.


### Test 4: Liste des Tools disponibles pour l'agent :

In [23]:
# Test 4: Outils à disposition
response = agent.invoke({
    "messages": [{"role": "user", "content": "quels sont les tools à ta disposition ?"}]
})
print(response['messages'][-1].content)

J'ai accès aux outils suivants :

1. **UrlSummarizer** : Cet outil permet de récupérer et de résumer le contenu principal d'une page web en fournissant une URL. Il peut également limiter le résumé à un certain nombre de mots.

2. **Calculator** : Cet outil évalue des expressions mathématiques. Par exemple, il peut calculer des opérations comme '12*(7+3)'.

Si tu as besoin d'aide avec l'un de ces outils, fais-le moi savoir !


In [24]:
# Test 5: Requête combinée + affichage des outils utilisés :
response = agent.invoke({
    "messages": [{"role": "user", "content": "Calcule l'âge du capitaine Némo au moment de sa disparition dans L'île Mystérieuse + 12 et ensuite résume cette page: https://comparia.beta.gouv.fr/, Donne une liste des tools utilisés pour répondre aux questions précédents"}]
})
print(response['messages'][-1].content)


L'âge du capitaine Némo au moment de sa disparition dans "L'île Mystérieuse" est de 70 ans. En ajoutant 12, cela donne un total de 82 ans.

### Résumé de la page https://comparia.beta.gouv.fr/
Le site "compar:IA" est un comparateur d'IA conversationnelles qui permet aux utilisateurs de discuter avec deux IA de manière anonyme et d'évaluer leurs réponses. Les données échangées sont utilisées à des fins de recherche. Les utilisateurs peuvent interagir aussi longtemps qu'ils le souhaitent, et les modèles d'IA sont révélés à la fin de la discussion. Le site vise à sensibiliser les citoyens aux enjeux de l'IA générative et à encourager le développement de l'esprit critique.

### Liste des tools utilisés
1. **Calculator** - Pour calculer l'âge du capitaine Némo + 12.
2. **UrlSummarizer** - Pour résumer la page web fournie.


## Étape 6: Ajout de garde-fous (guardrails) et Gestion des échecs (fallbacks)

### Partie 1: Guardrails de sécurité

In [25]:
def http_get_safe(url: str, retries: int = 3, backoff: float = 1.5, timeout: int = 15) -> requests.Response:
    """Version sécurisée de http_get avec guardrails."""
    last_err = None

    for attempt in range(1, retries + 1):
        try:
            # Requête HEAD pour vérifier le type de contenu et la taille
            head_resp = requests.head(url, timeout=5, headers={"User-Agent": "AgenticAI/1.0"}, allow_redirects=True)

            # Vérification du Content-Type
            content_type = head_resp.headers.get('Content-Type', '')
            if not (content_type.startswith('text/') or 'html' in content_type):
                raise ValueError(f"Type de contenu non supporté: {content_type}")

            # Vérification de la taille (max 2MB)
            content_length = head_resp.headers.get('Content-Length')
            if content_length and int(content_length) > 2 * 1024 * 1024:
                raise ValueError(f"Page trop volumineuse: {int(content_length)/1024/1024:.2f}MB")

            # Requête GET complète
            r = requests.get(url, timeout=timeout, headers={"User-Agent": "AgenticAI/1.0"})
            r.raise_for_status()
            return r

        except requests.RequestException as e:
            last_err = e
            if attempt == retries:
                raise
            time.sleep(backoff ** attempt)

    raise last_err

# Mettre à jour fetch_and_extract pour utiliser la version sécurisée
cache_safe = TTLCache(maxsize=256, ttl=60 * 10)

@cached(cache_safe)
def fetch_and_extract_safe(url: str) -> str:
    """Version sécurisée avec guardrails."""
    resp = http_get_safe(url)
    return extract_main_text(resp.text)

print("✓ Guardrails de sécurité ajoutés:")
print("  - Vérification du Content-Type (text/* ou html)")
print("  - Limite de taille: 2MB")
print("  - Blocage des fichiers binaires")

✓ Guardrails de sécurité ajoutés:
  - Vérification du Content-Type (text/* ou html)
  - Limite de taille: 2MB
  - Blocage des fichiers binaires


In [34]:
import time

# Test 1: URL valide (page HTML normale)
print("=" * 60)
print("TEST 1: URL valide - Page HTML")
print("=" * 60)
try:
    url = "https://example.com"
    response = http_get_safe(url)
    print(f"✓ Succès pour {url}")
    print(f"  - Status code: {response.status_code}")
    print(f"  - Content-Type: {response.headers.get('Content-Type')}")
    print(f"  - Taille: {len(response.content)} bytes")
except Exception as e:
    print(f"✗ Erreur: {e}")

print("\n")

# Test 2: Extraction de texte avec fetch_and_extract_safe
print("=" * 60)
print("TEST 2: Extraction de texte principal")
print("=" * 60)
try:
    url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
    text = fetch_and_extract_safe(url)
    print(f"✓ Texte extrait de {url}")
    print(f"  - Longueur du texte: {len(text)} caractères")
    print(f"  - Aperçu: {text[:200]}...")
except Exception as e:
    print(f"✗ Erreur: {e}")

print("\n")

# Test 3: Fichier binaire (devrait échouer)
print("=" * 60)
print("TEST 3: Fichier PDF - Devrait échouer")
print("=" * 60)
try:
    url = "https://www.w3.org/WAI/ER/tests/xhtml/testfiles/resources/pdf/dummy.pdf"
    response = http_get_safe(url)
    print(f"✗ Ne devrait pas réussir: {url}")
except ValueError as e:
    print(f"✓ Bloqué correctement: {e}")
except Exception as e:
    print(f"✗ Erreur inattendue: {e}")

print("\n")

# Test 4: URL invalide (devrait utiliser les retries)
print("=" * 60)
print("TEST 4: URL invalide - Test des retries")
print("=" * 60)
try:
    url = "https://this-url-does-not-exist-12345.com"
    start_time = time.time()
    response = http_get_safe(url, retries=2)
    print(f"✗ Ne devrait pas réussir")
except Exception as e:
    elapsed = time.time() - start_time
    print(f"✓ Échec attendu après {elapsed:.2f}s: {type(e).__name__}")

print("\n")

# Test 5: Cache test (devrait être instantané la 2ème fois)
print("=" * 60)
print("TEST 5: Test du cache")
print("=" * 60)
try:
    url = "https://example.com"

    # Première requête
    start = time.time()
    text1 = fetch_and_extract_safe(url)
    time1 = time.time() - start

    # Deuxième requête (depuis le cache)
    start = time.time()
    text2 = fetch_and_extract_safe(url)
    time2 = time.time() - start

    print(f"✓ Première requête: {time1:.3f}s")
    print(f"✓ Deuxième requête (cache): {time2:.3f}s")
    print(f"✓ Accélération: {time1/time2:.1f}x plus rapide")
    print(f"✓ Textes identiques: {text1 == text2}")
except Exception as e:
    print(f"✗ Erreur: {e}")

print("\n")

# Test 6: Test avec l'agent
print("=" * 60)
print("TEST 6: Utilisation avec l'agent")
print("=" * 60)
try:
    # Créer un outil utilisant la version sécurisée
    from langchain_core.tools import Tool

    def summarize_url_safe(url: str, max_words: int = 120):
        """Résume une URL de manière sécurisée."""
        text = fetch_and_extract_safe(url)
        words = text.split()[:max_words]
        return ' '.join(words) + '...'

    safe_url_tool = Tool(
        name="SafeUrlSummarizer",
        func=summarize_url_safe,
        description="Récupère et résume une page web de manière sécurisée (max 2MB, HTML uniquement)."
    )

    # Test direct de l'outil
    result = safe_url_tool.func("https://example.com")
    print(f"✓ Résumé généré:")
    print(f"  {result[:150]}...")

except Exception as e:
    print(f"✗ Erreur: {e}")

TEST 1: URL valide - Page HTML
✓ Succès pour https://example.com
  - Status code: 200
  - Content-Type: text/html
  - Taille: 513 bytes


TEST 2: Extraction de texte principal
✓ Texte extrait de https://en.wikipedia.org/wiki/Artificial_intelligence
  - Longueur du texte: 89744 caractères
  - Aperçu: Artificial intelligence ( AI ) is the capability of computational systems to perform tasks typically associated with human intelligence , such as learning , reasoning , problem-solving , perception , ...


TEST 3: Fichier PDF - Devrait échouer
✓ Bloqué correctement: Type de contenu non supporté: application/pdf; qs=0.001


TEST 4: URL invalide - Test des retries
✓ Échec attendu après 1.54s: ConnectionError


TEST 5: Test du cache
✓ Première requête: 0.106s
✓ Deuxième requête (cache): 0.000s
✓ Accélération: 8866.0x plus rapide
✓ Textes identiques: True


TEST 6: Utilisation avec l'agent
✓ Résumé généré:
  This domain is for use in documentation examples without needing permission. Avoid use 

### Partie 2: Raffinement avec LLM (optionnel)

In [27]:
from openai import OpenAI

client = OpenAI()

def refine_with_llm(text: str, max_words: int = 120) -> str:
    """Raffine le résumé en utilisant un LLM."""
    prompt = f"Résume le texte suivant en {max_words} mots maximum, en préservant les faits clés:\n\n{text}"

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return resp.choices[0].message.content.strip()

def summarize_url_enhanced(url: str, max_words: int = 120, use_llm: bool = True) -> str:
    """Version améliorée avec guardrails et raffinement LLM."""

    # Validation
    try:
        args = UrlSummarizeInput(url=url, max_words=max_words)
    except ValidationError as ve:
        return f"Erreur de validation: {ve}"

    # Récupération sécurisée
    try:
        text = fetch_and_extract_safe(str(args.url))
        if not text or len(text.split()) < 30:
            return "Désolé, impossible de trouver suffisamment de contenu lisible."
    except Exception as e:
        return f"Échec: {e}"

    # Résumé
    if use_llm and len(text.split()) > max_words:
        # Utiliser le LLM pour un résumé de meilleure qualité
        try:
            # Limiter le texte d'entrée pour éviter les tokens excessifs
            truncated = " ".join(text.split()[:500])
            return refine_with_llm(truncated, max_words)
        except Exception as e:
            print(f"Avertissement: Échec du raffinement LLM ({e}), utilisation de l'heuristique")

    # Fallback: heuristique simple
    words = text.split()
    if len(words) <= args.max_words:
        return " ".join(words)

    out = []
    count = 0
    for sentence in re.split(r'(?<=[.!?])\s+', text):
        sw = sentence.split()
        if count + len(sw) > args.max_words:
            break
        out.append(sentence)
        count += len(sw)

    if not out:
        return " ".join(words[:args.max_words]) + " ..."

    return " ".join(out)

print("✓ Version améliorée avec raffinement LLM créée")

# Test comparatif
print("\nTest avec example.com (sans LLM):")
result1 = summarize_url_enhanced("https://1data-hexagone.fr/", max_words=50, use_llm=False)
print(f"Résultat: {result1}")

print("\nTest avec example.com (avec LLM):")
result2 = summarize_url_enhanced("https://comparia.beta.gouv.fr/", max_words=50, use_llm=True)
print(f"Résultat: {result2}")

✓ Version améliorée avec raffinement LLM créée

Test avec example.com (sans LLM):
Résultat: MaÃ®trisez les technologies leaders du Big Data Chez 1Data-Hexagone, nous formons les professionnels aux Ã©cosystÃ¨mes Hadoop, aux plateformes de streaming en temps rÃ©el, et aux solutions IA agentiques modernes. Des formations pratiques, orientÃ©es mÃ©tier, par des experts confirmÃ©s.

Test avec example.com (avec LLM):
Résultat: Le comparateur d'IA compar:IA permet aux utilisateurs de discuter avec deux IA anonymes pour évaluer leurs réponses. Cet outil gratuit sensibilise à l'IA générative et collecte des données pour améliorer les modèles. Il est soutenu par le Ministère de la Culture et vise à rendre l'IA plus accessible et transparente.


## Étape 7: (Optionnel) Exposition en tant qu'outil OpenAI Function-Calling tool

Version native avec Function Calling d'OpenAI.

In [29]:
import json

# Schéma JSON pour Function Calling
url_summarizer_schema = {
    "type": "function",
    "function": {
        "name": "summarize_url",
        "description": "Récupère et résume le contenu d'une page web",
        "parameters": {
            "type": "object",
            "properties": {
                "url": {
                    "type": "string",
                    "description": "L'URL de la page web à résumer"
                },
                "max_words": {
                    "type": "integer",
                    "description": "Nombre maximum de mots dans le résumé",
                    "default": 120
                }
            },
            "required": ["url"]
        }
    }
}

def run_function_calling_agent(user_query: str):
    """Agent utilisant Function Calling natif d'OpenAI."""

    messages = [{"role": "user", "content": user_query}]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=[url_summarizer_schema],
        tool_choice="auto"
    )

    response_message = response.choices[0].message

    # Vérifier si le modèle veut appeler une fonction
    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "summarize_url":
                # Extraire les arguments
                function_args = json.loads(tool_call.function.arguments)
                print(f"\n🔧 Appel de fonction: {tool_call.function.name}")
                print(f"   Arguments: {function_args}")

                # Exécuter la fonction
                result = summarize_url_enhanced(**function_args)
                print(f"   Résultat: {result[:100]}...\n")

                # Ajouter le résultat à la conversation
                messages.append(response_message)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result
                })

                # Obtenir la réponse finale
                final_response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages
                )

                return final_response.choices[0].message.content

    return response_message.content

# Test
print("✓ Agent Function Calling créé\n")
test_query = "Résume en 60 mots le contenu de https://1data-hexagone.fr/"
print(f"Query: {test_query}")
result = run_function_calling_agent(test_query)
print(f"\nRéponse finale: {result}")

✓ Agent Function Calling créé

Query: Résume en 60 mots le contenu de https://1data-hexagone.fr/

🔧 Appel de fonction: summarize_url
   Arguments: {'url': 'https://1data-hexagone.fr/', 'max_words': 60}
   Résultat: 1Data-Hexagone propose des formations pratiques sur les technologies Big Data, incluant Hadoop, solu...


Réponse finale: 1Data-Hexagone propose des formations pratiques sur le Big Data, incluant Hadoop, l'intelligence artificielle et le streaming en temps réel. Dirigées par des experts, les sessions couvrent la gestion des données massives, les bases NoSQL et les requêtes analytiques. Elles sont adaptées aux entreprises, avec un accompagnement post-formation et un accès à des environnements lab.


## Étape 8: Pour aller plus loin :

### Idées d'extensions possibles:

1. **Filtres de métadonnées:**
   - Ajouter des allowlist/denylist de domaines
   - Filtrer par langue du contenu

2. **Rate-limiting et cache avancé:**
   - Implémenter un rate-limiter par domaine
   - Cache persistant avec Redis ou diskcache
   - Cache partagé entre plusieurs agents

3. **Sources et RAG:**
   - Retourner des métadonnées (titre, auteur, date)
   - Extraire et retourner des snippets cités
   - Format compatible RAG avec embeddings

4. **Observabilité:**
   - Logs structurés
   - Métriques (latence, cache hit rate)
   - Tracing distribué

5. **Robustesse:**
   - Support JavaScript rendering (Playwright/Selenium)
   - Extraction de PDFs
   - Support de l'authentification

6. **LangChain Runnable:**
   - Convertir en Runnable pour LCEL
   - Chaînage avec d'autres outils
   - Support async/streaming

### Exemple d'extension: Filtrage de domaines

In [33]:
from urllib.parse import urlparse
from typing import List, Optional

class DomainFilter:
    """Filtre pour gérer les listes d'autorisation/blocage de domaines."""

    def __init__(self, allowlist: Optional[List[str]] = None, denylist: Optional[List[str]] = None):
        self.allowlist = set(allowlist or [])
        self.denylist = set(denylist or [])

    def is_allowed(self, url: str) -> tuple[bool, str]:
        """Vérifie si l'URL est autorisée."""
        domain = urlparse(url).netloc

        # Si allowlist existe, le domaine doit y être
        if self.allowlist and domain not in self.allowlist:
            return False, f"Domaine non autorisé: {domain}"

        # Vérifier la denylist
        if domain in self.denylist:
            return False, f"Domaine bloqué: {domain}"

        return True, f"OK pour : {domain}"

def summarize_url_filtered(url: str, max_words: int = 120,
                           domain_filter: Optional[DomainFilter] = None) -> str:
    """Version avec filtrage de domaines."""

    # Vérification du domaine
    if domain_filter:
        allowed, message = domain_filter.is_allowed(url)
        if not allowed:
            return f"Accès refusé: {message}"

    return summarize_url_enhanced(url, max_words)

# Exemple d'utilisation
filter_example = DomainFilter(
    allowlist=["cloud.google.com", "openai.com", "github.com"],
    denylist=["pirateproxy.space"]
)

print("✓ Système de filtrage de domaines créé\n")

# Tests
print("Test 1 - Domaine autorisé:")
print(filter_example.is_allowed("https://cloud.google.com/discover/"))

print("\nTest 2 - Domaine bloqué:")
print(filter_example.is_allowed("https://www.pirateproxy.space/"))

print("\nTest 3 - Domaine non dans allowlist:")
print(filter_example.is_allowed("https://1data-hexagone.fr/"))

✓ Système de filtrage de domaines créé

Test 1 - Domaine autorisé:
(True, 'OK pour : cloud.google.com')

Test 2 - Domaine bloqué:
(False, 'Domaine non autorisé: www.pirateproxy.space')

Test 3 - Domaine non dans allowlist:
(False, 'Domaine non autorisé: 1data-hexagone.fr')


## 🎉 Conclusion

Vous avez maintenant créé un outil personnalisé production-ready avec:

✅ **Validation de schéma** avec Pydantic  
✅ **Gestion d'erreurs** robuste avec retries  
✅ **Système de cache** avec TTL  
✅ **Guardrails de sécurité** (type/taille de contenu)  
✅ **Raffinement LLM** optionnel  
✅ **Intégration LangChain** et Function Calling natif  
✅ **Extensions** (filtrage de domaines)  

### Prochaines étapes:

1. Implémenter les extensions suggérées
2. Ajouter des tests unitaires et d'intégration
3. Déployer comme microservice avec FastAPI
4. Intégrer dans un système RAG complet
5. Ajouter monitoring et observabilité

**Bravo! Vous maîtrisez maintenant la création d'outils personnalisés pour agents!** 🚀